<h1>Analysis of a specific matchup<h1>

0. Imports

In [1]:
import pandas as pd
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import nba_analysis_helpers as methods
from plotly.subplots import make_subplots
from nba_api.stats.endpoints import leaguedashteamstats

<h2>Team parameters<h2>

In [2]:
team_1 = "Atlanta Hawks"
team_2 = "Sacramento Kings"

<h2>Make prepared logs for the teams<h2>

In [3]:
log_1 = methods.prepare_data(methods.get_team_log(team_1))
log_2 = methods.prepare_data(methods.get_team_log(team_2))


<h3>Here some plot about te teams ponts:<h3>

In [4]:
methods.make_hist_subplot(log_1,log_2,team_1,team_2,'PTS')

In [5]:
log_1['MATCHUP'] = methods.clean_matchup(log_1)
log_2['MATCHUP'] = methods.clean_matchup(log_2)

In [6]:
log_1_by_opponents = methods.scores_by_opponent(log_1, team_2)
log_2_by_opponents = methods.scores_by_opponent(log_2, team_1)

KeyError: 'SAC'

In [ ]:
from IPython.display import display_html 


df1_styler = log_1_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df1')
df2_styler = log_2_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df2')
df2_t_styler = log_2_by_opponents.T.style.set_table_attributes("style='display:inline'").set_caption('df2_t')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
# display_html(df1_styler._repr_html_()+df2_styler._repr_html_() +df2_t_styler._repr_html_(), raw=True)

,SAC
count,2.000000
min,117.000000
max,137.000000
mean,127.000000
median,127.000000
,LAL
count,2.000000
min,92.000000
max,141.000000
mean,116.500000


<h2>Lets see the season statics ot the two teams<h2>

In [7]:
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_2_ID = teams.find_teams_by_full_name(team_2)[0]['id']

teams_stats = pd.DataFrame(
    [leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_1_ID).get_normalized_dict()['LeagueDashTeamStats'][0],
    leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_2_ID).get_normalized_dict()['LeagueDashTeamStats'][0]]
    )

teams_stats[['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'FG_PCT', 'FG3_PCT','FT_PCT', 'PLUS_MINUS', 'CFID' ]]

,TEAM_NAME,GP,W,L,W_PCT,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,CFID
0,Atlanta Hawks,36,16,20,0.444,0.462,0.380,0.807,-2.0,10
1,Sacramento Kings,39,16,23,0.410,0.454,0.344,0.761,-167.0,10


<h3>Lets visualize it:<h3>

In [8]:
methods.make_stat_comparison_subplot(teams_stats,team_1,team_2)

<h2>Lets make a train dataset to make a ML model to predict games<h2>

<h3>Table from the team stats by each game<h3>

In [9]:
from nba_api.stats.endpoints import teamgamelog
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_games = teamgamelog.TeamGameLog(team_id=team_1_ID).get_normalized_dict()["TeamGameLog"]

team_games = pd.DataFrame(team_games)
print(team_games.columns)
team_games.head()

Index(['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612737,0022100560,"JAN 03, 2022",ATL @ POR,L,16,20,0.444,240,48,...,0.875,9,30,39,31,6,1,12,24,131
1,1610612737,0022100533,"DEC 31, 2021",ATL @ CLE,W,16,19,0.457,240,43,...,0.905,14,30,44,23,8,4,2,23,121
2,1610612737,0022100519,"DEC 29, 2021",ATL @ CHI,L,15,19,0.441,240,44,...,0.692,10,27,37,26,6,3,11,17,117
3,1610612737,0022100502,"DEC 27, 2021",ATL vs. CHI,L,15,18,0.455,240,40,...,0.885,11,34,45,22,3,5,10,14,118
4,1610612737,0022100488,"DEC 25, 2021",ATL @ NYK,L,15,17,0.469,240,33,...,0.571,8,38,46,23,8,5,12,12,87


In [10]:
#címke tömb
labels = team_games['WL']

#ebbe a táblába csak a statokat rakjuk
train_stats = team_games[['W', 'L', 'W_PCT']]

#hozzá kell adni egy plusz sort az elejére
train_stats = train_stats.append({'W':0,'L':0,'W_PCT':0}, ignore_index=True)
#meg kell fordítani a sorrendet
train_stats = train_stats[::-1]
#el kell dobni az utolsó sort
train_stats = train_stats.drop(0)
#indexeket a megfordítás miatt újra kell állítani
train_stats = train_stats.reset_index(drop=True)

#ide azokat amiket el kell csúsztatni
train = team_games[['Game_ID', 'GAME_DATE', 'MATCHUP']][::-1]
#indexeket a megfordítás miatt újra kell állítani
train = train.reset_index(drop=True)

#össze kell rakni a két elkészült táblákat 
train = pd.concat([train,train_stats], axis=1)

#át kell alakítani a matchupot úgy hogy értelmesebb, használhatóbb legyen
train['MATCHUP'] = train['MATCHUP'].map(lambda x: x.split(' ')[2])

train['GAME_DATE'] = pd.to_datetime(train['GAME_DATE'], format="%b %d, %Y")
train['day_in_month'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%d")))
train['day_in_week'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%w")))
train = train.drop('GAME_DATE', axis=1)
train.head()

,Game_ID,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,0022100014,DAL,0,0,0.000,21,4
1,0022100027,CLE,1,0,1.000,23,6
2,0022100043,DET,1,1,0.500,25,1
3,0022100059,NOP,2,1,0.667,27,3
4,0022100066,WAS,3,1,0.750,28,4


<h3>Table from the team lineups<h3>

<h4>Model trained by the team stats<h4>

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.3, random_state=12)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=2,random_state=12)
model.fit(X_train,y_train)

model

ValueError: could not convert string to float: 'PHX'